<a href="https://colab.research.google.com/github/markusloecher/DataScience2021/blob/main/TWSM/class7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install transformers
!pip install datasets transformers[sentencepiece]


https://huggingface.co/course/chapter3/2?fw=tf

In [9]:
import numpy as np
import pandas as pd
#possibly replace with e.g. util.pytorch_cos_sim from sentence_transformers
from scipy.spatial.distance import cosine

from transformers import pipeline


In [5]:
#this will download the default gpt2 model, around 1.2GB (or just 530MB??)
generator = pipeline("text-generation")
#sex bias?
generator("She was very ")

No model was supplied, defaulted to gpt2 (https://huggingface.co/gpt2)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'She was very icky while eating. I couldn\'t stand her."\n\n"No, you can come stay at the hotel." She said, leaning over one arm to hug him. "We\'ll stay there through the cold."\n\n"'}]

In [4]:
generator("He was very ")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'He was very \xa0ambitious and the product was highly sought after.\xa0 He was successful in Japan though it seems his country had little demand and he was sold on the market.\xa0 He was a very well paid man with a very impressive resume'}]

Mask filling

In [7]:
unmasker = pipeline("fill-mask")
print(unmasker("She works as a <mask> .", top_k=2))
print(unmasker("He works as a <mask> .", top_k=2))

No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)


[{'score': 0.08421123772859573, 'token': 35698, 'token_str': ' waitress', 'sequence': 'She works as a waitress.'}, {'score': 0.06919262558221817, 'token': 28894, 'token_str': ' translator', 'sequence': 'She works as a translator.'}]
[{'score': 0.06483733654022217, 'token': 8298, 'token_str': ' consultant', 'sequence': 'He works as a consultant.'}, {'score': 0.06146613135933876, 'token': 28894, 'token_str': ' translator', 'sequence': 'He works as a translator.'}]


In [9]:
print(unmasker("The child ran to her <mask> .", top_k=4))
print(unmasker("The child ran to his <mask> .", top_k=4))

[{'score': 0.23610961437225342, 'token': 985, 'token_str': ' mother', 'sequence': 'The child ran to her mother.'}, {'score': 0.11077290028333664, 'token': 1150, 'token_str': ' father', 'sequence': 'The child ran to her father.'}, {'score': 0.06188420578837395, 'token': 3795, 'token_str': ' mom', 'sequence': 'The child ran to her mom.'}, {'score': 0.05995608866214752, 'token': 1041, 'token_str': ' parents', 'sequence': 'The child ran to her parents.'}]
[{'score': 0.2751142680644989, 'token': 985, 'token_str': ' mother', 'sequence': 'The child ran to his mother.'}, {'score': 0.16092269122600555, 'token': 1150, 'token_str': ' father', 'sequence': 'The child ran to his father.'}, {'score': 0.09510596841573715, 'token': 1041, 'token_str': ' parents', 'sequence': 'The child ran to his parents.'}, {'score': 0.06645991653203964, 'token': 8853, 'token_str': ' grandmother', 'sequence': 'The child ran to his grandmother.'}]


### Transfer Learning

Training a pre traind BERT model on 2 sentences !

In [7]:
import tensorflow as tf
import numpy as np
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = dict(tokenizer(sequences, padding=True, truncation=True, return_tensors="tf"))

# This is new
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
labels = tf.convert_to_tensor([1, 1])
model.train_on_batch(batch, labels)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


7.8842387199401855

### Tasks

1. Load the bing sentiment dictionary
2. Divide into 50% training and 50% test
3. Retrain a BERT model with train
4. Predict on test
5. Try GPU instead of CPU

In [10]:
from google.colab import drive
drive.mount('/content/drive')

TWSM_path = "/content/drive/MyDrive/teaching/TWSM/WorkInClass/"
bing = pd.read_csv(TWSM_path+"bing.csv")

bing

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,word,sentiment
0,1,2-faces,negative
1,2,abnormal,negative
2,3,abolish,negative
3,4,abominable,negative
4,5,abominably,negative
...,...,...,...
6781,6782,zealously,negative
6782,6783,zenith,positive
6783,6784,zest,positive
6784,6785,zippy,positive
